# 3 Cancer de seno.

Divide aleatoriamente el conjunto de datos de cáncer de seno de Wisconsin 3 en un subconjunto de
entrenamiento con el 70 % de los datos y un subconjunto de validación con el 30 % restante usando 0
como semilla para tu generador de números aleatorios. Este conjunto de datos contiene 699 registros
de tumores de seno, de los cuales 458 son benignos y 241 son malignos. Cada registro consta de los
siguientes atributos:

| Numero  | Atributo  | Valores | 
| ------- |:-------------:|:-------------:| 
| 1       | Código de la muestra |  ID |
| 2       | Grosor del tumor|  1-10 |
| 3       | Uniformidad del tamaño de la célula |  1-10 |
| 4       | Uniformidad de la forma de la célula |  1-10 |
| 5       | Adhesión marginal|  1-10 |
| 6       | Tamaño de célula epitelial |  1-10  |
| 7       | Núcleos desnudos |  1-10 |
| 8       | Cromatina blanda|  1-10 |
| 9       | Nucléolos normales |  1-10  |
| 10      | Mitosis de células | 1-10 |
| 11      | Clase|  2 para benigno ; 4 para maligno

Entrena un clasificador bayesiano ingenuo de tumores de seno y evalúalo tanto con el subconjunto
de entrenamiento como con el subconjunto de validación y discute su desempeño. Existen 16 registros
en el conjunto de datos con un atributo no especificado. Investiga estrategias para rellenar los datos
faltantes, utiliza las que consideres más adecuadas para este problema y discute el impacto en el
desempeño del clasificador.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score

cancer = pd.read_csv('breast-cancer-wisconsin.data',header=None,sep=',')
Cancer = pd.DataFrame(cancer)
Cancer.rename(columns={0:'ID',1:'G',2:'UT',3:'UF',4:'A',5:'T',6:'ND',7:'Cr',8:'NN',9:'M',10:'C'}, inplace=True)
Cancer.head(8)

,ID,G,UT,UF,A,T,ND,Cr,NN,M,C
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2


In [2]:
#Imprimimos la informacion del dataframe para ver posibles ubicaciones de los datos faltantes
print(Cancer.info())
#La info es muy util ya que se observa que todas las columnas excepto la 6 o ND son de tipo numerico
#Esto indica que ND es del tipo string y ahi es donde deben estar los datos faltantes

#buscamos en la columna ND del dataframe el caracter ? (despues de ver el dataframe sabemos que eso denota el dato faltante)
#esta buysqueda se hace con la funcion contains()
Perdidos = Cancer[Cancer['ND'].str.contains('\?')]
print(Perdidos)

indicesPerdidos = Cancer[Cancer['ND'].str.contains('\?')].index
print(indicesPerdidos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      699 non-null    int64 
 1   G       699 non-null    int64 
 2   UT      699 non-null    int64 
 3   UF      699 non-null    int64 
 4   A       699 non-null    int64 
 5   T       699 non-null    int64 
 6   ND      699 non-null    object
 7   Cr      699 non-null    int64 
 8   NN      699 non-null    int64 
 9   M       699 non-null    int64 
 10  C       699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB
None
          ID  G  UT  UF  A  T ND  Cr  NN  M  C
23   1057013  8   4   5  1  2  ?   7   3  1  4
40   1096800  6   6   6  9  6  ?   7   8  1  2
139  1183246  1   1   1  1  1  ?   2   1  1  2
145  1184840  1   1   3  1  2  ?   2   1  1  2
158  1193683  1   1   2  1  3  ?   1   1  1  2
164  1197510  5   1   1  1  2  ?   3   1  1  2
235  1241232  3   1   4  1  2  ?   3   1 

Vemos que las filas de los datos faltantes hay 2 que son cancer y 14 que no lo son, esta proporcion puede desnivelar las predicciones favoreciendo al cancer. Esto se discute mas adelante. Por lo pronto supondremos que esos datos nunca existieron.

In [3]:
Cancer1 = pd.DataFrame(Cancer.drop(indicesPerdidos))
Cancer1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      683 non-null    int64 
 1   G       683 non-null    int64 
 2   UT      683 non-null    int64 
 3   UF      683 non-null    int64 
 4   A       683 non-null    int64 
 5   T       683 non-null    int64 
 6   ND      683 non-null    object
 7   Cr      683 non-null    int64 
 8   NN      683 non-null    int64 
 9   M       683 non-null    int64 
 10  C       683 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 64.0+ KB


In [4]:
indicesPerdidos1 = Cancer1[Cancer1['ND'].str.contains('\?')].index
indicesPerdidos1

Int64Index([], dtype='int64')

In [5]:
Cancer1['ND'] = pd.to_numeric(Cancer1['ND'])
Cancer1

,ID,G,UT,UF,A,T,ND,Cr,NN,M,C
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [6]:
#Definimos nuestros conjuntos de entrenamiento y validacion
#train se construye a partir de la funcion sample() donde frac es el porcentaje de los datos
#random_state es la semilla del numero aleatorio, esta funcion por default tiene desactivado
#el ingreso de datos por reemplazo, es decir no se pueden repetir filas
#el conjunto test se construye a partir del train quitando todos los indices que tiene train,
#es decir hace una operacion de conjuntos del conjunto universo Spam resta el conjunto train 
#y guarda lo que resta en test
train = Cancer1.sample(frac=0.7,random_state=0)
test = Cancer1.drop(train.index)
#reseteamos los indices con la funcion reset_index. drop=True lo usamos para evitar que se
#anada una columna del indice anterior
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

Train = pd.DataFrame(train)
Test = pd.DataFrame(test)

In [7]:
#creamos las matrices numpy X,Y de entrenamiento y test
X_train = pd.DataFrame(Train.loc[:,'G':'M']).to_numpy()
Y_train = pd.DataFrame(Train[['C']]).to_numpy()


X_test = pd.DataFrame(Test.loc[:,'G':'M']).to_numpy()
Y_test = pd.DataFrame(Test[['C']]).to_numpy()

In [8]:
print('El conjunto de entrenamiento tiene %d muestras.' %X_train.shape[0])
print('El conjunto de validacion tiene %d muestras.' %X_test.shape[0])

El conjunto de entrenamiento tiene 478 muestras.
El conjunto de validacion tiene 205 muestras.


## Con una distribucion Categorica para todos los eventos:

In [9]:
#Ajustamos a un clasificador Naive Bayes de distribuciones Categoricas
Cat = CategoricalNB()
Cat.fit(X_train,Y_train.ravel())

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
#predecimos las salidas de los datos de entrenamiento con nuestro modelo
Y_pred_train = Cat.predict(X_train)
Y_pred_train

array([2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 2, 2, 2,
       4, 2, 4, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2,
       2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4,
       4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 2, 2, 4, 2, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 2, 4, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 2, 4, 2, 4, 4, 2, 2,
       2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 2,
       2, 4, 2, 2, 4, 4, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 4, 4, 2, 2, 2, 2, 4, 4, 2, 2,
       4, 4, 2, 2, 4, 4, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2,
       2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2,
       4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4,
       2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 2, 4,

In [11]:
#Imprimimos las salidas reales
print(Y_train.T)

[[2 2 4 4 2 2 2 4 2 2 4 2 4 2 2 2 4 4 4 2 2 2 4 2 4 4 2 2 2 4 2 4 4 2 2 2
  4 4 2 4 2 2 2 2 2 2 2 4 2 2 4 2 4 2 2 2 4 2 2 4 2 2 2 2 2 2 2 2 4 4 2 2
  2 2 2 2 4 2 2 2 4 2 4 2 2 4 2 2 4 2 4 2 4 4 4 2 4 4 4 2 2 2 4 4 2 2 4 4
  2 2 4 2 2 4 2 2 2 4 2 2 2 4 2 2 4 4 2 4 2 4 2 2 4 2 2 4 2 4 2 2 2 4 2 2
  2 2 4 2 4 2 4 4 2 2 2 2 4 4 2 4 4 4 4 4 4 2 2 2 2 2 2 4 4 4 2 2 2 4 2 2
  4 4 2 4 2 2 4 4 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 4 4 2 4 2 2 2 4 2 2 2 2
  4 4 2 2 4 4 2 2 4 4 2 4 2 4 4 2 2 2 4 2 4 2 4 2 2 2 2 2 4 2 2 4 2 2 4 2
  2 2 2 2 4 2 2 4 2 4 2 2 4 4 4 2 2 2 2 2 2 4 4 2 2 2 2 2 2 2 4 2 2 4 2 2
  2 2 2 2 2 4 2 2 2 4 4 2 4 2 2 2 4 2 2 2 4 4 2 4 2 2 4 2 2 2 2 2 2 2 4 4
  4 4 2 4 2 4 2 4 4 4 2 2 4 2 2 2 2 4 4 2 2 2 4 2 2 4 2 2 2 2 4 4 2 2 2 2
  2 2 2 2 2 2 2 4 2 4 2 2 2 2 2 2 4 4 4 4 2 2 4 2 4 2 4 2 2 2 2 4 2 4 2 2
  2 2 4 2 2 4 2 2 2 2 2 2 2 4 2 4 2 2 2 4 2 2 2 2 2 4 2 2 2 2 4 2 2 4 2 2
  2 2 4 4 2 2 2 2 4 2 2 4 2 2 2 2 4 4 2 4 2 4 2 2 2 4 4 4 2 2 2 2 2 2 2 2
  4 4 2 2 2 2 2 2 2 4]]


In [12]:
#Imprimimos las estadisticas
print('La exactitud relativa del modelo es %.5f, para la prediccion de los datos de entrenamiento' %accuracy_score(Y_train,Y_pred_train))
print('La cantidad de errores de un total de %d muestras es : %d' % (X_train.shape[0], (Y_train.T != Y_pred_train).sum()))
print('Estos errores representan el %.2f porciento' %(100*((Y_train.T != Y_pred_train).sum()/X_train.shape[0] )))

La exactitud relativa del modelo es 0.97280, para la prediccion de los datos de entrenamiento
La cantidad de errores de un total de 478 muestras es : 13
Estos errores representan el 2.72 porciento


In [13]:
#Predecimos las salidas de los datos de validacion con nuestro modelo y las imprimimos
Y_pred_test = Cat.predict(X_test)
Y_pred_test

array([2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4,
       2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 2, 2, 4, 2,
       2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 4, 2,
       2, 4, 2, 2, 4, 4, 4, 2, 4, 4, 4, 4, 2, 4, 2, 4, 2, 4, 2, 4, 4, 4,
       2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 2, 2,
       4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2,
       2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 2, 2, 2, 4,
       2, 2, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2,
       4, 2, 2, 2, 2, 2, 2], dtype=int64)

In [14]:
#Imprimimos la salida real
print(Y_test.T)

[[2 2 2 2 4 2 2 2 2 2 2 4 2 4 4 2 4 4 4 4 4 4 2 2 4 2 2 4 4 2 2 2 2 2 2 4
  4 4 4 4 2 2 4 2 2 2 4 2 2 2 4 2 4 4 2 4 2 2 4 2 4 4 4 2 4 2 2 4 2 2 4 4
  4 4 4 4 2 4 2 4 2 2 2 4 2 4 4 4 2 4 4 4 4 4 2 4 4 4 2 4 2 4 2 2 2 4 4 2
  2 2 4 2 4 2 2 4 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 4 2 2 4 2 2 2 2 4 2 4 4
  2 2 2 4 2 2 4 2 4 2 2 4 2 2 4 2 2 2 2 2 2 2 2 4 2 4 4 4 2 2 2 4 2 2 4 2
  4 2 2 4 2 2 2 2 2 2 4 2 2 2 2 2 2 2 4 2 2 2 2 2 2]]


In [15]:
#Imprimimos las estadisticas
print('La exactitud relativa del modelo es %.5f, para la prediccion de los datos de validacion' %accuracy_score(Y_test,Y_pred_test))
print('La cantidad de errores de un total de %d muestras es : %d' % (X_test.shape[0], (Y_test.T != Y_pred_test).sum()))
print('Estos errores representan el %.2f porciento' %(100*((Y_test.T != Y_pred_test).sum()/X_test.shape[0] )))

La exactitud relativa del modelo es 0.97561, para la prediccion de los datos de validacion
La cantidad de errores de un total de 205 muestras es : 5
Estos errores representan el 2.44 porciento


## Con una Multinomial para todos los eventos:

In [16]:
#Modelamos nuestro naive bayes con una multinomial y ajustamos datos
Multi = MultinomialNB()
Multi.fit(X_train,Y_train.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
#predecimos las salidas de los datos de entrenamiento con nuestro modelo
Y_pred_train_multi = Multi.predict(X_train)
Y_pred_train_multi

array([4, 2, 4, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2,
       4, 2, 4, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 2, 4, 2, 2, 2, 2, 2, 2,
       2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4,
       4, 2, 4, 2, 4, 2, 4, 4, 4, 4, 2, 2, 2, 2, 4, 4, 2, 4, 4, 2, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4, 4, 4, 4, 2, 4, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2,
       2, 2, 4, 4, 2, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 2,
       2, 4, 2, 2, 4, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 2, 2, 2, 4, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4, 2, 2,
       2, 4, 2, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 2,
       2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2,
       4, 4, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 4, 2, 2, 4,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 4,

In [18]:
#Imprimimos las estadisticas
print('La exactitud relativa del modelo es %.5f, para la prediccion de los datos de entrenamiento' %accuracy_score(Y_train,Y_pred_train_multi))
print('La cantidad de errores de un total de %d muestras es : %d' % (X_train.shape[0], (Y_train.T != Y_pred_train_multi).sum()))
print('Estos errores representan el %.2f porciento' %(100*((Y_train.T != Y_pred_train_multi).sum()/X_train.shape[0] )))

La exactitud relativa del modelo es 0.89958, para la prediccion de los datos de entrenamiento
La cantidad de errores de un total de 478 muestras es : 48
Estos errores representan el 10.04 porciento


In [19]:
#Predecimos las salidas de los datos de validacion con nuestro modelo y las imprimimos
Y_pred_test_multi = Multi.predict(X_test)
Y_pred_test_multi

array([2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 4, 4, 4, 2, 4, 2,
       2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 2,
       2, 2, 4, 2, 2, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 4, 4, 4, 2, 4, 2,
       2, 4, 2, 2, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 2, 4, 2, 2, 2, 4, 2, 2,
       2, 4, 4, 4, 4, 4, 2, 2, 4, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2,
       4, 2, 4, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 2, 4, 2,
       2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 4, 4, 4, 2, 2, 2, 4,
       2, 2, 4, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [20]:
#Imprimimos las estadisticas
print('La exactitud relativa del modelo es %.5f, para la prediccion de los datos de validacion' %accuracy_score(Y_test,Y_pred_test_multi))
print('La cantidad de errores de un total de %d muestras es : %d' % (X_test.shape[0], (Y_test.T != Y_pred_test_multi).sum()))
print('Estos errores representan el %.2f porciento' %(100*((Y_test.T != Y_pred_test_multi).sum()/X_test.shape[0] )))

La exactitud relativa del modelo es 0.91220, para la prediccion de los datos de validacion
La cantidad de errores de un total de 205 muestras es : 18
Estos errores representan el 8.78 porciento


Vemos que asumiendo distribucion Categorica obtenemos mejores resultados que asumir una multinomial. Ya que la cantidad de error de la prediccion de los datos de prueba y validacion es mucho menor.
Por otro lado nuestro clasificador con Categoricas tiene un buen rendimiento en ambos conjuntos de datos, obteniendo un porcentaje de error menor al 3% en ambos casos.

## Estimacion de valores perdidos.

Ya vimos que todos los datos faltantes corresponden a la columna de nucleos desnudos. Proponemos dos formas de estimarlos. La primera de ellas consiste en separar los datos entre maligno y benigno, y obtener las medias para cada par de conjuntos y con ese dato rellenar los datos faltantes.
La otra consiste en separar los datos entre maligno y benigno, observar la correlacion entre la variable nucleos desnudos y las otras 8 variables, para despues realizar una estimacion lineal y rellenar los valores segun el dato de la otra variable al ingresar los valores de dicha variable en la regresion lineal.
Las suposiciones fuertes son que la variable nucleos desnudos es continua y suave. 

### Division de los conjuntos

In [21]:
#Sobre la columna 'C' (de cancer) buscamos los valores que contengan el numero 2,
#es decir que sean benignos y lo hacemos un dataframe
Benigno = pd.DataFrame(Cancer.loc[Cancer['C'] == 2])

#Sobre la columna 'C' (de cancer) buscamos los valores que contengan el numero 4,
#es decir que sean malignos y lo hacemos un dataframe
Maligno = pd.DataFrame(Cancer.loc[Cancer['C'] == 4])

print('Hay %d muestras de benigno y %d muestras de maligno. Ya lo sabiamos e.e' %(Benigno.shape[0],Maligno.shape[0]))

Hay 458 muestras de benigno y 241 muestras de maligno. Ya lo sabiamos e.e


### Promedios y valores esperados

In [22]:
indices_ben = Benigno[Benigno['ND'].str.contains('\?')].index
indices_mal = Maligno[Maligno['ND'].str.contains('\?')].index


media_ben = pd.DataFrame(Benigno.drop(indices_ben))
media_ben['ND'] = pd.to_numeric(media_ben['ND'])
media_ben = media_ben['ND'].mean()

media_mal = pd.DataFrame(Maligno.drop(indices_mal))
media_mal['ND'] = pd.to_numeric(media_mal['ND'])
media_mal = media_mal['ND'].mean()

print(media_ben,media_mal)
print(indices_ben)

1.3468468468468469 7.627615062761507
Int64Index([40, 139, 145, 158, 164, 235, 249, 275, 294, 297, 315, 321, 411,
            617],
           dtype='int64')


In [23]:
Benigno['ND'] = np.where((Benigno.ND == '?'),int(media_ben),Benigno.ND)
Maligno['ND'] = np.where((Maligno.ND == '?'),int(media_mal),Maligno.ND)

In [25]:
Benigno['ND'].head(40)

0      1
1     10
2      2
3      4
4      1
6     10
7      1
8      1
9      1
10     1
11     1
13     3
16     1
17     1
19     1
22     1
24     1
26     1
27     1
28     1
29     1
30     1
31     1
33     1
34     1
35     1
37     1
40     1
45     1
47     1
48     1
61     2
64     1
66     1
69     1
70     1
72     1
75     2
76     1
77     1
Name: ND, dtype: object